In [1]:
import os
import re
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

def parse_week(s):
  pattern = re.compile(r"Week (?P<number>\d+): (?P<qty>\d)/(?P<total>\d)")
  match = pattern.match(s)

  number = int(match.group("number"))
  qty = int(match.group("qty"))
  total = int(match.group("total"))

  return (number, qty, total)

directory = 'Export-ab37a8aa-56cd-4e32-a09b-96b05c412542/Habit Tracker de9812bc2ef24f37abe36f13c9109c47/'

files = os.listdir(directory)
habits = []
for filename in files:
  lines = []
  with open(directory + filename) as file:
      try:
          lines = file.readlines()
      except:
          print(filename)

  name = ""
  weeks = []
  for line in lines:
    if line.startswith("# "):
      name = line.replace("# ", "").strip()
    if line.startswith("Week "):
      weeks.append(parse_week(line.strip()))

  for week in weeks:
    habits.append([name, week[0], week[1], week[2]])

df = pd.DataFrame(habits, columns=['name', 'week', 'done', 'total'])
df['result'] = np.floor(df['done'] / df['total'] * 100)
df = df.sort_values(by="week")

start_end = df.groupby('name').agg({'week': ['min', 'max']})

habits_names = df['name'].unique()

fig = make_subplots(
  rows=6, cols=4, subplot_titles=habits_names, shared_yaxes=True,
  shared_xaxes=True, vertical_spacing=0.04, horizontal_spacing=0.01)

i = 0
means = []
for habit_name in habits_names:
  row = (i // 4) + 1
  col = (i % 4) + 1
  habit = df.query("name=='%s'" % habit_name)
  fig.add_trace(go.Scatter(x=habit["week"], y=habit["result"],
                            mode='lines+markers', name='', text=habit["done"]),
                row=row, col=col)
  i += 1
  means.append({"i": i, "mean": int(np.floor(habit["result"].mean()))})

fig.update_xaxes(range=[0.5, 52])
fig.update_yaxes(range=[-10, 110])
shapes = list(map(lambda x: dict(
  x0=0, x1=52, y0=x["mean"], y1=x["mean"], xref='x' + str(x["i"]), yref='y' + str(x["i"]),
  line_width=1, line_color="green" if x["mean"] > 50 else "red"), means))
means_annotations = list(map(lambda x: dict(
  x=52, y=x["mean"] + 15, xref='x' + str(x["i"]), yref='y' + str(x["i"]),
  showarrow=False, xanchor='right', text=x["mean"]), means))

fig.update_layout(showlegend=False, shapes=shapes, margin_t=20, margin_b=20, margin_r=20, margin_l=20, height=1500)

annotations = [a.to_plotly_json() for a in fig["layout"]["annotations"]]
annotations.extend(means_annotations)
fig["layout"]["annotations"] = annotations

fig.show(config={'displayModeBar': False})
fig.write_image("results.png")